In [20]:
import tensorflow_hub as hub

module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"


bert_layer = hub.KerasLayer(module_url, trainable=True)

In [2]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/dataset_efrei_test/questions.csv")
# train=train.head(10000)
print(len(train))
train.head(3)

404351


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0


In [3]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [4]:
!pip install sentencepiece &> /dev/null

In [23]:
from transformers import BertTokenizer
import numpy as np

# Charger le tokenizer de BERT depuis transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []

    for text in texts:
        # Tokenisation du texte
        tokens = tokenizer.tokenize(text)

        tokens = tokens[:max_len-2]  # Limiter la longueur du texte
        input_sequence = ["[CLS]"] + tokens + ["[SEP]"]
        pad_len = max_len - len(input_sequence)

        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len  # Remplir avec des zéros pour obtenir une longueur max_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len  # Masque de padding
        segment_ids = [0] * max_len  # Les IDs de segment pour BERT

        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)


In [24]:
# train_input_r = bert_encode(train.question1.values, tokenizer, max_len=100)
# train_input_l = bert_encode(train.question2.values, tokenizer, max_len=100)
# y = train.is_duplicate.values

In [5]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.layers import Input, LSTM, Dense, Layer
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np

# Charger le tokenizer et le modèle BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Définir une couche personnalisée pour BERT
class BertLayer(Layer):
    def __init__(self, bert_model, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert_model = bert_model

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        return outputs.last_hidden_state

# Définir une couche personnalisée pour LSTM
class LSTMWrapper(Layer):
    def __init__(self, units, **kwargs):
        super(LSTMWrapper, self).__init__(**kwargs)
        self.lstm = LSTM(units, return_sequences=False)

    def call(self, inputs):
        return self.lstm(inputs)

# Définir une couche personnalisée pour l'expansion des dimensions
class ExpandDims(Layer):
    def __init__(self, axis, **kwargs):
        super(ExpandDims, self).__init__(**kwargs)
        self.axis = axis

    def call(self, inputs):
        return tf.expand_dims(inputs, axis=self.axis)

# Définir une couche personnalisée pour la distance de Manhattan
class ManhattanDistance(Layer):
    def call(self, inputs):
        left, right = inputs
        return tf.exp(-tf.reduce_sum(tf.abs(left - right), axis=1, keepdims=True))

# Définir le modèle avec BERT et LSTM
class SiameseBertLSTM:
    def __init__(self, max_seq_length=50):
        self.max_seq_length = max_seq_length

    def build_model(self, bert_model):
        # Définir les entrées
        input_ids_left = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='input_ids_left')
        attention_mask_left = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='attention_mask_left')
        token_type_ids_left = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='token_type_ids_left')

        input_ids_right = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='input_ids_right')
        attention_mask_right = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='attention_mask_right')
        token_type_ids_right = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='token_type_ids_right')

        # Appliquer BERT via la couche personnalisée
        bert_layer = BertLayer(bert_model)
        bert_output_left = bert_layer([input_ids_left, attention_mask_left, token_type_ids_left])
        bert_output_right = bert_layer([input_ids_right, attention_mask_right, token_type_ids_right])

        # Extraire le [CLS] token pour la classification
        clf_output_left = bert_output_left[:, 0, :]
        clf_output_right = bert_output_right[:, 0, :]

        # Passer par une LSTM partagée
        lstm_wrapper = LSTMWrapper(50)

        # Ajouter une dimension pour les timesteps
        expand_dims_layer = ExpandDims(axis=1)
        clf_output_left_expanded = expand_dims_layer(clf_output_left)
        clf_output_right_expanded = expand_dims_layer(clf_output_right)

        left_lstm_output = lstm_wrapper(clf_output_left_expanded)
        right_lstm_output = lstm_wrapper(clf_output_right_expanded)

        # Calculer la distance de Manhattan entre les deux sorties LSTM
        distance_output = ManhattanDistance()([left_lstm_output, right_lstm_output])

        # Créer le modèle final
        model = Model(inputs=[input_ids_left, attention_mask_left, token_type_ids_left,
                              input_ids_right, attention_mask_right, token_type_ids_right],
                      outputs=[distance_output])

        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
        return model

# Exemple d'utilisation avec des textes simples
siamese_model = SiameseBertLSTM(max_seq_length=50)
model = siamese_model.build_model(bert_model=bert_model)

# Préparer des données fictives pour tester
def prepare_input(texts, tokenizer, max_len=50):
    inputs = tokenizer(texts, padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')
    return (inputs['input_ids'].numpy(),  # Convertir en numpy array
            inputs['attention_mask'].numpy(),  # Convertir en numpy array
            inputs['token_type_ids'].numpy())  # Convertir en numpy array

def prepare_data(texts_left, texts_right, labels, tokenizer, max_len=50):
    input_ids_left, attention_mask_left, token_type_ids_left = prepare_input(texts_left, tokenizer, max_len)
    input_ids_right, attention_mask_right, token_type_ids_right = prepare_input(texts_right, tokenizer, max_len)
    y = np.array(labels, dtype=np.float32)  # Convertir en numpy array
    return (input_ids_left, attention_mask_left, token_type_ids_left,
            input_ids_right, attention_mask_right, token_type_ids_right), y

# Textes et étiquettes fictifs
texts_left = train.question1.values.tolist()
texts_right = train.question2.values.tolist()
labels = train.is_duplicate.values.tolist()  # Labels fictifs
def get_valid_indices(texts_left, texts_right):
    """
    Retourne les indices des éléments valides dans texts_left et texts_right.
    """
    valid_indices = [i for i in range(len(texts_left))
                     if isinstance(texts_left[i], str) and isinstance(texts_right[i], str)]
    return valid_indices
valid_indices = get_valid_indices(texts_left, texts_right)
def filter_data(texts_left, texts_right, labels, valid_indices):
    texts_left_filtered = [texts_left[i] for i in valid_indices]
    texts_right_filtered = [texts_right[i] for i in valid_indices]
    labels_filtered = [labels[i] for i in valid_indices]
    return texts_left_filtered, texts_right_filtered, labels_filtered

texts_left_filtered, texts_right_filtered, labels_filtered = filter_data(texts_left, texts_right, labels, valid_indices)

inputs, y = prepare_data(texts_left_filtered, texts_right_filtered, labels_filtered, tokenizer, max_len=50)

# Diviser les données en ensembles d'entraînement et de test
(X_train_input_ids_left, X_test_input_ids_left,
 X_train_attention_mask_left, X_test_attention_mask_left,
 X_train_token_type_ids_left, X_test_token_type_ids_left,
 X_train_input_ids_right, X_test_input_ids_right,
 X_train_attention_mask_right, X_test_attention_mask_right,
 X_train_token_type_ids_right, X_test_token_type_ids_right,
 y_train, y_test) = train_test_split(
    inputs[0],  # input_ids_left
    inputs[1],  # attention_mask_left
    inputs[2],  # token_type_ids_left
    inputs[3],  # input_ids_right
    inputs[4],  # attention_mask_right
    inputs[5],  # token_type_ids_right
    y,
    test_size=0.2,
    random_state=42
)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# Entraîner le modèle
model.fit(
    [X_train_input_ids_left, X_train_attention_mask_left, X_train_token_type_ids_left,
     X_train_input_ids_right, X_train_attention_mask_right, X_train_token_type_ids_right],
    y_train,
    epochs=5,
    batch_size=2048,
    validation_data=(
        [X_test_input_ids_left, X_test_attention_mask_left, X_test_token_type_ids_left,
         X_test_input_ids_right, X_test_attention_mask_right, X_test_token_type_ids_right],
        y_test
    )
)

Epoch 1/5
